In [1]:
import chromadb

In [2]:
import numpy as np

In [3]:
pip show chromadb

Name: chromadb
Version: 0.5.5
Summary: Chroma.
Home-page: 
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: C:\Users\shree\.conda\envs\playing_with_genai\Lib\site-packages
Requires: bcrypt, build, chroma-hnswlib, fastapi, grpcio, httpx, importlib-resources, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-instrumentation-fastapi, opentelemetry-sdk, orjson, overrides, posthog, pydantic, pypika, PyYAML, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = ""
#Set api key in environmental variable,,thus while calling model ,, intrnally referred...........OPENAI_API_KEY is special/predefined meaning,,dont change name

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings #Although i am using Word2Vec pretrained model instead openai's embedding model,,u can use any other pretrained embedding model like BERT,GPT..etc,,,,just to practice..LOL
from langchain.llms import OpenAI #not going to use it,,bcz not have money,,,just written for practice
from langchain.document_loaders import DirectoryLoader , TextLoader #DirectoryLoader -> mention path for directory,,and it will read all files from it , TextLoader -> mention ,txt file path and it will load that file

## 1) Load The Documents (.txt, .pdf , .csv ,....etc)

In [5]:
loader = DirectoryLoader("Resume_Data" , glob="./*.txt" , loader_cls=TextLoader)
#Read all .txt files from specified directory,,,this directory has pdf , txt , word(.docx) and csv files,,,,but our code considered just txt file

In [6]:
fetched_data = loader.load()

In [7]:
fetched_data 
#When you read data using pypdf,document loader ..etc class,,then it will give you list like below
#[Document(metadata={'source': 'Resume_Data\\How_Billing_works_in_OpenAI.txt'}, page_content=.......] 1 document..............same conventions is used whenever you read the data using pypdf,document loader..
#each page data is represented in page_content variable......

#when you do chunking,,then also same convention is used to represent each chunk ... ie; data is present in page_content....ie;in this format/convention only -> [Document(metadata={'source': 'Resume_Data\\How_Billing_works_in_OpenAI.txt'}, page_content=.............
#Thus all langchain method will take this raw document,,and internally access page_content,so not need to manually

[Document(metadata={'source': 'Resume_Data\\How_Billing_works_in_OpenAI.txt'}, page_content='####NOTE-> There is no recharge kind of things,as u do for mobile recharge like 299 for 1 month,,,,,,,,,,,,,,,,,,,,,,,,,\n##in openai this kind of thing is not there,,where you have to to pay x amount and you will get y tokens........no\n#just give credit card details -> use any models you want ->  open ai will track , which model you are using,whats is the tokens rate for it..etc -> and after completeting billing cycle(generally 1 month is there) tou get invoice and payment is deducted from your card,,,-> if card has not enough money,,then you are temporarily suspened,,untill you pay there charges ,,,untill that you are not able to use there model.......\n\n##############################################################################################################\n1st) you have to add debit card details(ecommerce and international transaction should be enabled ie;VISA )\n2nd) you can add $5

In [8]:
fetched_data[0].page_content

'####NOTE-> There is no recharge kind of things,as u do for mobile recharge like 299 for 1 month,,,,,,,,,,,,,,,,,,,,,,,,,\n##in openai this kind of thing is not there,,where you have to to pay x amount and you will get y tokens........no\n#just give credit card details -> use any models you want ->  open ai will track , which model you are using,whats is the tokens rate for it..etc -> and after completeting billing cycle(generally 1 month is there) tou get invoice and payment is deducted from your card,,,-> if card has not enough money,,then you are temporarily suspened,,untill you pay there charges ,,,untill that you are not able to use there model.......\n\n##############################################################################################################\n1st) you have to add debit card details(ecommerce and international transaction should be enabled ie;VISA )\n2nd) you can add $5 to $100 in your openai account,,,,,,,,,,after billing cycle or 1 month amount is deducted f

## 2) breakdown above data into sentences or paragraphs or chunks,

- bcz many models process 4096 tokens or words at a time like gpt 3.5 turbo...etc or per minute it can process x amount of tokens only,,,,,in that caseif u give any data which has more tokens than it can process at a time then it will not process it,,,so for that case we have to pass input in chunk
- **if data is very-very huge,,then you cant pass this data directly to any Gen AI model (llm/lim/mm and Embedding model),,,we have to do chunking and pass it**
- bcz every gen ai model and embedding model has some limitation over token processing,,,,,so this technique will help in those case
- **1 document means 1 sentence, documents means set or collection of sentence**,,,see "document_list"

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000 , chunk_overlap=20)
#each chunk or sentence or paragraph has on an average 1000 tokens or words in it............not exactlt 1000 tokencan come in chunk
#last 20 tokens or word of i th chunk is starting of i+1 th chunk,,,,,,,,,,,,,,,,,,,,for i=1,,there is no previous chunk so,,no repeataion
#ie;last 20 tokens or word of i th chunk is carried to i+1 th chunk
#and also , we use chunk_overlap to sustain context or info of previous chunk

In [11]:
text = text_splitter.split_documents(fetched_data )

In [12]:
text
#each sentence/chunk is represented as 1 document ....
#[Document(metadata={'source': 'Resume_Data\\How_Billing_works_in_OpenAI.txt'}, page_content=.........] -> 1 document or 1 sentence or 1 chunk

[Document(metadata={'source': 'Resume_Data\\How_Billing_works_in_OpenAI.txt'}, page_content='####NOTE-> There is no recharge kind of things,as u do for mobile recharge like 299 for 1 month,,,,,,,,,,,,,,,,,,,,,,,,,\n##in openai this kind of thing is not there,,where you have to to pay x amount and you will get y tokens........no\n#just give credit card details -> use any models you want ->  open ai will track , which model you are using,whats is the tokens rate for it..etc -> and after completeting billing cycle(generally 1 month is there) tou get invoice and payment is deducted from your card,,,-> if card has not enough money,,then you are temporarily suspened,,untill you pay there charges ,,,untill that you are not able to use there model.......'),
 Document(metadata={'source': 'Resume_Data\\How_Billing_works_in_OpenAI.txt'}, page_content='##############################################################################################################\n1st) you have to add debit card det

In [13]:
text[1].page_content

'##############################################################################################################\n1st) you have to add debit card details(ecommerce and international transaction should be enabled ie;VISA )\n2nd) you can add $5 to $100 in your openai account,,,,,,,,,,after billing cycle or 1 month amount is deducted from it,,w.r.t usage of tokens in total\n3rd) you can add threshold,,,when your openai amount is lesser than this then automatically money is debitted from your account,,,,,,,,,,,,,,,ie;you have to provide minimum and maximum amount should present in you account as threshold\n\nBilling Process Overview\nCredit Card Setup\n\nEntry: Provide your credit card details in OpenAIâ€™s billing settings.\nVerification: OpenAI verifies the card for payment processing.\nUsage Tracking'

In [14]:
len(text)
#5 chunks are formed

5

In [15]:
document_list=list()
for i in range(len(text)):
    document_list.append(text[i].page_content)
document_list

['####NOTE-> There is no recharge kind of things,as u do for mobile recharge like 299 for 1 month,,,,,,,,,,,,,,,,,,,,,,,,,\n##in openai this kind of thing is not there,,where you have to to pay x amount and you will get y tokens........no\n#just give credit card details -> use any models you want ->  open ai will track , which model you are using,whats is the tokens rate for it..etc -> and after completeting billing cycle(generally 1 month is there) tou get invoice and payment is deducted from your card,,,-> if card has not enough money,,then you are temporarily suspened,,untill you pay there charges ,,,untill that you are not able to use there model.......',
 '##############################################################################################################\n1st) you have to add debit card details(ecommerce and international transaction should be enabled ie;VISA )\n2nd) you can add $5 to $100 in your openai account,,,,,,,,,,after billing cycle or 1 month amount is deducted

## 3) Creating Chroma DB

- **instead of openai embeddings i am using Word2Vec embeddings bcz it is opensource**

In [16]:
persist_directory = 'my_chroma_db' #Name or path ,,where we are going to store our embeddings related to text
#created directory name "my_chroma_db" in your current working directory and inside that sql3lite named file is there,,,,,,,,inside that your vectors are going to stored
#as u see we have not mentioned any complete path,,,,,thus in current working directory "vector_db" folder/directory is created and inside that,,,chroma.sqllite3 lite file is created,,and here only our vectors are stored
#Persistent means permanent

#### a) How OpenAI Embeddings and chroma db Works

In [30]:
#FOR STORING VECTORS?EMBEDDINGS INTO CHROMA DB (my_chroma_db)

# openai_embedding_model = OpenAIEmbeddings()
# vector_db = Chroma.from_documents(
#     documents = text,
#     embedding = embedding,
#     persist_directory = persist_directory
# )

#If u use openai embedding model,,,,,,,then u can run above code easily,,,,,,,,,,,,,,,,bcz 
#this "openai_emnedding_model" will take 0ne document or text from documents parameter -> embedding.embed_query(text[i].page_content) -> get this the vector/embeddings for this sentence or document
#and each vector is store in vector_db name database along with it's equvivalent text/document,,so at the time of retrieval it will return you just document,,no vector....but pinecone returns you vector,,and you have again revert it to text/document

#from_documents() -> (present in langchain),is implemented such that ,, we have to just pass the document (see ,,text,,,same we pass it without any preprocessing)
#ie;no need to,extract page_content,store in list and then pass it to embedding model,,,like what we done in "document_list"............
#from_documents(),,implemented such that,,it will internally extract page_content and pass it to further stage.........ie; to our embedding model one by one


In [ ]:
# TO ACCESS CHROMA DB (my_chroma_db),,,TO PERFORM QUERY OR QA TASK.............ypu can access my_chroma_db using above specified vector_db also,,or using below code also(ie;through retriever object vectordb).

# vectordb = Chroma(
#     persist_directory = "my_chroma_db", embedding_function=embedding              
# )

# retriever = vectordb.as_retriever()

# docs = retriever.get_relevant_documents("how OpenAI tracks your usage of API?")

# docs

- **Entire Process** looks like this->
- 1st- from_documents() will extract page_content from 1st document from text ,
- 2nd- pass it to embedding model,in above case it is openai embeddings model,,where it is implemented such that,it can directly take passed document(page_content) and internally do tokenization->vectorization for each token->aggregate all words vectors to get single vector which represent passed document or page_content,and return this vector............NOTE-> extracted document is passed to openai server,there only this vectorization is done using there embedding model and after that openai will return vector w.r.t passed document...SO internet should be on,,,and **you have to pay for this service**.........this **openai embedding object has embed_documents() mehtod in it,,which internally calls openai embedding API**,,,,thus if we are using word2vec or ..etc local side embedding model then we have to implment this kind of mechanism using class to access embedding model from local side,,**see below i have implemented this mechanism for accessing local side embedding model**
- 3rd- from_documents()  will take  this returned vector and store it in **my_chroma_db -> chroma.sqllite3**
- and these 3 process is repeated untill all documents from text is processed
- and finally from_documents() will return **vector_db object or pointer**,,through which we can access my_chroma_db to perform QA or query on stored vectors or embeddings
- **but we cant do same thing with w2v,bcz we have manually do word tokenization -> converting each word into vector -> aggregating all words vector to form single vector which represent the passed document or sentence,,,so we have to write some code manually**

#### b) Doing same thing with Word2Vec,,,which is downloaded pretrained embedding model

##### b.1) Incorrect way

In [17]:
from gensim.models import Word2Vec,KeyedVectors
path = r"D:\GENERATIVE-AI\004_VectorDB\Word2Vec_Model_300\GoogleNews-vectors-negative300.bin.gz"
w2v_embedding_model = KeyedVectors.load_word2vec_format(path, binary=True)

In [18]:
# Convert documents to embeddings
def document_to_embedding(doc, model):
    words = doc.split()
    word_vectors = [model[word] for word in words if word in model]
    if not word_vectors:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# Create embeddings for all documents
embeddings = [document_to_embedding(doc, w2v_embedding_model) for doc in document_list]

In [19]:
embeddings

[array([ 4.20454070e-02,  1.54501870e-02,  5.30535057e-02,  1.00846082e-01,
        -6.73474446e-02,  4.49416004e-02,  6.63796365e-02, -5.88520318e-02,
         4.44916524e-02,  5.55446893e-02, -6.68885782e-02, -6.30467087e-02,
        -5.31262644e-02, -5.25721116e-03, -7.93680325e-02,  1.16814777e-01,
         9.85055342e-02,  7.44783357e-02,  1.46714188e-02, -2.76448317e-02,
        -6.18413351e-02,  9.28246137e-03,  4.58155386e-02,  2.25885911e-03,
         2.54410300e-02,  4.93529160e-03, -1.02348655e-01,  5.60542792e-02,
         2.09426880e-03, -8.96844640e-03, -3.97800356e-02, -3.04915267e-03,
        -2.99420711e-02, -1.03935618e-02, -2.25857995e-03, -2.52478067e-02,
         1.48092126e-02,  2.58752778e-02,  2.40009595e-02,  4.30617929e-02,
         7.35564828e-02, -3.53977866e-02,  1.30084619e-01, -3.62702794e-02,
        -4.06904938e-03, -7.17198327e-02, -2.71792524e-03,  1.29923476e-02,
         1.42405443e-02, -1.68489590e-02, -1.67753641e-02,  6.15480021e-02,
        -3.7

In [20]:
# vector_db = Chroma.from_documents(
#     documents = text,
#     embedding = embeddings,
#     persist_directory = persist_directory
# )

- above code gives you error,bcz
- The error occurs because Chroma expects an embedding function, not precomputed embeddings. Let's modify the approach to provide a custom embedding function that Chroma can use to compute embeddings on the fly.
- **so we have to implement embed_document() method which calls this word2vec model**
- **We will define a custom embedding class that integrates with Chroma. This class should have an embed_documents method that Chroma can call.**

##### b.2) Correct Way

- **NOTE-> from_documents(),,will store vector/embeddings along with their equvivalent document/text**,,thus, **when u query something(using retriever.get_relevant_documents()),,then first that query is vectorized and then passed it to chroma db,,there it will do similarity matching b/n queried vector(vector of question asked) and other stored vectors,,,and return more similar Document.....(not return the vector,,,,,this is the beautiful thing of chroma db,,,,so no need to revert back vector to text/document again)**

In [21]:
# Convert documents to embeddings
def document_to_embedding(doc, model):
    words = doc.split()
    word_vectors = [model[word] for word in words if word in model]
    if not word_vectors:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0).tolist()

# Custom embedding class
class Word2VecEmbeddings:
    def __init__(self, model):
        self.model = model

    def embed_documents(self, docs): #"from_documents()" ..method will need this function to create embedding while storing in to vector db
        return [document_to_embedding(doc, self.model) for doc in docs]

    def embed_query(self, query): #when you query/question something using "retriever.get_relevant_documents()",,,then ,first you have to convert it to vectors then pass it to chroma db for similarity matching,,,,,,and most similar document is returned
        return document_to_embedding(query, self.model)


In [22]:
word2vec_embedding = Word2VecEmbeddings(w2v_embedding_model)

In [23]:
vector_db = Chroma.from_documents(
    documents=text,
    embedding=word2vec_embedding,
    persist_directory=persist_directory
)

In [24]:
# Optionally, you can persist the vector database to disk
vector_db.persist()
#Stored successfully

C:\Users\shree\.conda\envs\playing_with_genai\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


## 4) Querying or Implementing QA System on stored Vectors/embeddings

- **Biggest Disadvantage with chroma db is,,it will store these vetor as binary files (BIN file).......so u cant see it,,as u see in pinecone**
- in backend,,**chroma db uses chroma.sqllite3**

### 4.1)Load the persisted DB from disk(path where you created this my_chroma_db)

In [26]:
vectordb = Chroma(
    persist_directory = "my_chroma_db", embedding_function=word2vec_embedding              
)
#embedding_function=word2vec_embedding -> compulsorly you have to pass it,,,bcz when you query something,,at that time ,,
#to convert that query to vector,,it will internally use this model,,and embed_query mehthod of this "word2vec_embedding" object

In [27]:
vectordb
#pointer or object to access my_chroma_db

### 4.2) Make a retriever

In [45]:
retriever = vectordb.as_retriever(search_kwargs={'k':2})
#Top 2 most similar vectors are selected and return the corresponding document/text which is stored

In [46]:
#Question/query
docs = retriever.get_relevant_documents("how OpenAI tracks your usage of API?")

In [47]:
#Answer
docs

[Document(metadata={'source': 'Resume_Data\\How_Billing_works_in_OpenAI.txt'}, page_content='API Requests: OpenAI tracks your usage of API models based on the number of tokens processed or requests made.\nModel-Specific Pricing: Each model has its own token pricing (e.g., GPT-4o Mini).\nBilling Cycle\n\nCycle Duration: Typically monthly. Usage is tracked continuously throughout this period.\nAggregation: At the end of the cycle, total usage is aggregated.\nCharges and Payment\n\nInvoice Generation: An invoice is generated detailing your usage and charges.\nAutomatic Deduction: The total amount due is automatically deducted from your credit card. This usually happens at the end of the billing cycle or shortly thereafter.\nNo Per-Request Charges: You are not charged after each individual API request; charges accumulate over the billing cycle.\nHandling Payment Issues'),
 Document(metadata={'source': 'Resume_Data\\How_Billing_works_in_OpenAI.txt'}, page_content='API Requests: OpenAI track

In [48]:
len(docs)

2

In [49]:
#  1st most similar document
docs[0].page_content

'API Requests: OpenAI tracks your usage of API models based on the number of tokens processed or requests made.\nModel-Specific Pricing: Each model has its own token pricing (e.g., GPT-4o Mini).\nBilling Cycle\n\nCycle Duration: Typically monthly. Usage is tracked continuously throughout this period.\nAggregation: At the end of the cycle, total usage is aggregated.\nCharges and Payment\n\nInvoice Generation: An invoice is generated detailing your usage and charges.\nAutomatic Deduction: The total amount due is automatically deducted from your credit card. This usually happens at the end of the billing cycle or shortly thereafter.\nNo Per-Request Charges: You are not charged after each individual API request; charges accumulate over the billing cycle.\nHandling Payment Issues'

In [50]:
# 2nd most similar document
docs[1].page_content
#i dont know why it is giving same respoonse..
#but it is giving more accurate document only,as u see

'API Requests: OpenAI tracks your usage of API models based on the number of tokens processed or requests made.\nModel-Specific Pricing: Each model has its own token pricing (e.g., GPT-4o Mini).\nBilling Cycle\n\nCycle Duration: Typically monthly. Usage is tracked continuously throughout this period.\nAggregation: At the end of the cycle, total usage is aggregated.\nCharges and Payment\n\nInvoice Generation: An invoice is generated detailing your usage and charges.\nAutomatic Deduction: The total amount due is automatically deducted from your credit card. This usually happens at the end of the billing cycle or shortly thereafter.\nNo Per-Request Charges: You are not charged after each individual API request; charges accumulate over the billing cycle.\nHandling Payment Issues'

- as u see we asked **"how OpenAI tracks your usage of API?"**,,,,,and we get response which contains answer for this question at starting only ->   **" OpenAI tracks your usage of API models based on the number of tokens processed or requests made"**
- but i want a specific answer from retrieved document w.r.t question,,then use LLM for doing this task,,,,,,,,,,,where it will pass the returned document to LLM along with question,,,and llm will answer specifically,,,w/o giving thisbig document,,,,,,,,,,,
- **so we go for RetrievalQA concept**,,,for increasing interactiveness

### 4.3) RetrievalQA , for refining the response obtained from retriever object.......

- **Basic RAG implementations**

In [51]:
from langchain.chains import RetrievalQA

In [ ]:
llm = OpenAI() #U can use any model
#from environmental variable it will access,,API key for openai
#note-> we are not able to choose a specific model,,we get any of the default gpt model

In [ ]:
llm
#

In [ ]:
#Create chain to answer question
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever = retriever,
    return_source_document = True
)

In [ ]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print("\n\nsources")
    for source in llm_response["source_documents"]:
        print(source.metadata["source"])
        

In [ ]:
query = "charges for open AI model?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

- **RetrievalQA Process**ie;qa_chain will pass question to retriever object and retriever object will give the most similar document to question and this answer and question is then passed to llm and get specific answer

- See, we have not getting response from LLM for our question, we are using llm for just refinement process..ie;to get specific data from received document (which is most similar to asked question)
- you have 2 ways to do it,
 1) finetune llm with above data(ie;your usecase data),,and directly ask question to that llm without using any vector db,,,
 2) store your usecase data in database (sql , vector db(in the form of vectors/embeddings)),,and connect this database with your LLM,,,and ask question related to your data to LLM and it will answer it,,,,,,as we done above using RetrievalQA()..........and this 2nd way is known as **RAG(Retrieval Augmented Generation)**

- **We are getting Answers from vector db based on similarity b/n question vector (vector representation of question(text)) and answer vector,,,,,,,,,,
,,,but vector db returns you complete document which is desired answer and also which is most similar with question vector.................as u see above,,when i asked "how OpenAI tracks your usage of API?",,,then it gives me most similar document to this question vector,,,,,,which contains answer,,,,but it also contains unneccssory text also,,,,and i want a specific answer from this document,,then we use llm to parse it.ie; we pass this document to our llm model and it will give you specific answer to our question**

- and this process is called RAG

### Conclusion - LLM is just use for refining the answer from document we obtained from retriever....in RetrievalQA...which is RAG

## 5) Delete my_chroma_db database

In [ ]:
#Mehthod 1
vectordb.delete_collection()
vectordb.persist()#to persist changes

In [ ]:
#method 2
!zip -r my_chroma_db.zip ./my_chroma_db

In [ ]:
#method 3
!rm -rf my_chroma_db/